In [1]:
import json
import os

In [2]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return int(fragments[2])

In [3]:
path = "ExperimentData//"
path = "SampleSessionLayerableOnly//"
path = "1652473795203//"
path = "1652484257598//"
dir_list = os.listdir(path)

dir_list.sort(key=parseFileNumber)

print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)


Files and directories in ' 1652484257598// ' :
['RecordStudy_Session_0_1652484257598.json', 'RecordStudy_Session_1_1652484257598.json', 'RecordStudy_Session_2_1652484257598.json', 'RecordStudy_Session_3_1652484257598.json', 'RecordStudy_Session_4_1652484257598.json', 'RecordStudy_Session_5_1652484257598.json', 'RecordStudy_Session_6_1652484257598.json', 'RecordStudy_Session_7_1652484257598.json', 'RecordStudy_Session_8_1652484257598.json', 'RecordStudy_Session_9_1652484257598.json', 'RecordStudy_Session_10_1652484257598.json', 'RecordStudy_Session_11_1652484257598.json', 'RecordStudy_Session_12_1652484257598.json', 'RecordStudy_Session_13_1652484257598.json', 'RecordStudy_Session_14_1652484257598.json', 'RecordStudy_Session_15_1652484257598.json', 'RecordStudy_Session_16_1652484257598.json', 'RecordStudy_Session_17_1652484257598.json', 'RecordStudy_Session_18_1652484257598.json', 'RecordStudy_Session_19_1652484257598.json', 'RecordStudy_Session_20_1652484257598.json', 'RecordStudy_Sess

In [17]:
appEvents = []
codesEvents = []
experimentEvents = []

def parseData(json_data):
    frames = json_data["frames"]
    for frame in frames:
        appEvent = frame["appEvents"]
        codeEvent = frame["codesEvents"]
        experimentEvent = frame["experimentEvents"]
        if appEvent:
            appEvents.append(appEvent)
        if codeEvent:
            codesEvents.append(codeEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)


In [18]:

for filename in dir_list:
    data = []
    if filename.endswith(".json"):
        # Prints only text file present in My Folder
        #print(filename)
        firstFileFlag = False
        if "Session_0" in filename:
            firstFileFlag = True
        with open(path + filename, 'r') as f:
            json_data = json.load(f)
            if(firstFileFlag):
                #data.append(json_data["obj"]["sessionRecording"])
                parseData(json_data["obj"]["sessionRecording"])
            else:
                #data.append(json.load(f))
                parseData(json_data)

experiment event RecordStudy_Session_15_1652484257598.json
experiment event RecordStudy_Session_32_1652484257598.json
experiment event RecordStudy_Session_35_1652484257598.json
experiment event RecordStudy_Session_50_1652484257598.json


In [20]:
#print(appEvents)
#print(codesEvents)
print(experimentEvents[1])

[{'unixTime': 1652485371718, 'systemTime': '16-42-51-71', 'userID': '1652484257598', 'sessionType': 'immersive', 'eventName': 'EndTask', 'pinRandomSeed': 1312, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 6}]


In [6]:
#experimentEvents[0][0]
experimentEvents
pinCodes = experimentEvents[0][0]["pinCodes"]

for exper in experimentEvents:
    for exp in exper:
        if exp["eventName"] == "EndTask":
            print(exp)


{'unixTime': 1652485371718, 'systemTime': '16-42-51-71', 'userID': '1652484257598', 'sessionType': 'immersive', 'eventName': 'EndTask', 'pinRandomSeed': 1312, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 6}
{'unixTime': 1652486051838, 'systemTime': '16-54-11-83', 'userID': '1652484257598', 'sessionType': 'layerable', 'eventName': 'EndTask', 'pinRandomSeed': 1312, 'pinNumCodes': 200, 'pinCodeLength': 6, 'pinNumPieces': 3, 'pinCodes': [], 'score': 8}


In [7]:
import pandas
import itertools
pandas.options.display.max_rows = None

In [23]:
experimentEvents
experimentsUnpacked = list(itertools.chain(*experimentEvents))

experimentsDf = pandas.DataFrame(experimentsUnpacked)
experimentsDf



,unixTime,systemTime,userID,sessionType,eventName,pinRandomSeed,pinNumCodes,pinCodeLength,pinNumPieces,pinCodes,score
0,1652484771710,16-32-51-70,1652484257598,immersive,StartTask,1312,200,6,3,"[{'Value': 233429, 'Label': 'UW', 'Pieces': [{...",0
1,1652485371718,16-42-51-71,1652484257598,immersive,EndTask,1312,200,6,3,[],6
2,1652485451840,16-44-11-83,1652484257598,layerable,StartTask,1312,200,6,3,"[{'Value': 233429, 'Label': 'UW', 'Pieces': [{...",0
3,1652486051838,16-54-11-83,1652484257598,layerable,EndTask,1312,200,6,3,[],8


In [27]:
#check experiment events
immersiveStartTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'immersive') & (experimentsDf['eventName'] == 'StartTask')]['unixTime'].to_numpy()[0]
immersiveEndTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'immersive') & (experimentsDf['eventName'] == 'EndTask')]['unixTime'].to_numpy()[0]

layerableStartTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'layerable') & (experimentsDf['eventName'] == 'StartTask')]['unixTime'].to_numpy()[0]
layerableEndTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'layerable') & (experimentsDf['eventName'] == 'EndTask')]['unixTime'].to_numpy()[0]

# layerableCodeEntryEvents = codeEntryEvents.loc[(codeEntryEvents['unixTime'] > layerableStartTime) & (codeEntryEvents['unixTime'] < layerableEndTime)]
# layerableCodeEntryEvents

print("immersive time: " + str((immersiveEndTime - immersiveStartTime) / 1000 / 60))
print("layerable time: " + str((layerableEndTime - layerableStartTime) / 1000 / 60))

immersive time: 10.000133333333334
layerable time: 9.999966666666667


In [38]:
appEvents
appsUnpacked = list(itertools.chain(*appEvents))
appsDf = pandas.DataFrame(appsUnpacked)
appsDf.sort_values(by='eventTime')
#appsDf

,unixTime,systemTime,eventTime,activityID,activityType,activity,name
0,1652484284333,16-24-44-33,26735,62272f2e-9ddf-4fc4-8f7d-c0d4abb2d2a4,1,Start,Tutorial
1,1652484319973,16-25-19-97,62375,df60b774-9aff-4e19-b92a-bfc1d07a2439,1,Start,Code Entry
2,1652484336064,16-25-36-06,78466,f0b38703-976a-4c4e-ade8-4c808b4f61a6,1,Start,Item Inspector
3,1652484347754,16-25-47-75,90156,29f56202-8551-491e-be7d-dd9655062456,1,Start,Device Groups
4,1652484360027,16-26-00-02,102429,4eb9847b-2e07-450c-b1df-97f312036faf,1,Start,Atlas
5,1652484458205,16-27-38-20,200607,f0b38703-976a-4c4e-ade8-4c808b4f61a6,1,Stop,Item Inspector
6,1652484460765,16-27-40-76,203167,4eb9847b-2e07-450c-b1df-97f312036faf,1,Stop,Atlas
7,1652484461745,16-27-41-74,204147,df60b774-9aff-4e19-b92a-bfc1d07a2439,1,Stop,Code Entry
8,1652484462436,16-27-42-43,204838,29f56202-8551-491e-be7d-dd9655062456,1,Stop,Device Groups
9,1652484463345,16-27-43-34,205747,62272f2e-9ddf-4fc4-8f7d-c0d4abb2d2a4,1,Stop,Tutorial


In [46]:
immersiveEndTime = experimentsDf.loc[(experimentsDf['sessionType'] == 'immersive') & (experimentsDf['eventName'] == 'EndTask')]['unixTime'].to_numpy()[0]


tutorialEvents = appsDf[appsDf['name'] == 'Tutorial']
annotationEvents = appsDf[appsDf['name'] == 'AnnotationsMenu']
codeEntryEvents = appsDf[appsDf['name'] == 'Code Entry']
itemInspectorEvents = appsDf[appsDf['name'] == 'Item Inspector']
deviceGroupsEvents = appsDf[appsDf['name'] == 'Device Groups']
atlasEvents = appsDf[appsDf['name'] == 'Atlas']

atlasStarts = atlasEvents[atlasEvents['activity'] == 'Start']
atlasStops = atlasEvents[atlasEvents['activity'] == 'Stop']

tutorialStarts = tutorialEvents[tutorialEvents['activity'] == 'Start']
tutorialStops = tutorialEvents[tutorialEvents['activity'] == 'Stop']

tutorialStarts['timeDifference'] = tutorialStops.unixTime - tutorialStarts.unixTime
tutorialStarts



C:\Users\Lab\AppData\Local\Temp\ipykernel_13188\2023067717.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tutorialStarts['timeDifference'] = tutorialStops.unixTime - tutorialStarts.unixTime


,unixTime,systemTime,eventTime,activityID,activityType,activity,name,timeDifference
0,1652484284333,16-24-44-33,26735,62272f2e-9ddf-4fc4-8f7d-c0d4abb2d2a4,1,Start,Tutorial,NaN


In [45]:
tutorialStops

,unixTime,systemTime,eventTime,activityID,activityType,activity,name
9,1652484463345,16-27-43-34,205747,62272f2e-9ddf-4fc4-8f7d-c0d4abb2d2a4,1,Stop,Tutorial
